In [ ]:
"""
FNN-only pruning pipeline (auto-detect model type).
- Detects model type from final layer (binary / multiclass / regression).
- Adapts CSV loader accordingly.
- Keeps final layer intact (never pruned).
- Reports FLOPs, GFLOPs, MFLOPS and inference time.
"""

import numpy as np
import pandas as pd
import tensorflow as tf
import time
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
import os

# -----------------------
# Helper: detect model type
# -----------------------
def detect_model_type(model):
    out = model.layers[-1]
    units = getattr(out, "units", None)
    act = getattr(out, "activation", None)
    act_name = act.__name__ if act is not None else None

    if units == 1 and act_name in ("sigmoid",):
        return "binary"
    if units == 1 and act_name in ("linear", "relu", "tanh"):
        return "regression"
    if units is not None and units > 1 and act_name == "softmax":
        return "multiclass"
    # fallback tries
    if units == 1:
        return "binary"
    if units and units > 1:
        return "multiclass"
    return "unknown"

def get_loss_and_metrics_for_type(model_type):
    if model_type == "binary":
        return ("binary_crossentropy", ["accuracy"])
    if model_type == "multiclass":
        # use sparse labels to avoid forcing one-hot encoding
        return ("sparse_categorical_crossentropy", ["sparse_categorical_accuracy"])
    if model_type == "regression":
        return ("mse", ["mse"])
    raise ValueError("Unknown model type")

# =====================================================================
# 1) UNIVERSAL CSV LOADER  (adapts target based on model_type)
# =====================================================================
def load_any_csv_dataset(csv_path, target=None, test_size=0.2, val_size=0.1, model_type="binary"):
    """
    Loads CSV (auto-detect delimiter). Adapts target processing to model_type:
      - binary/regression: returns y as float32 shaped (n,1)
      - multiclass: returns integer labels shaped (n,1) for sparse loss
    """
    print("[INFO] Loading CSV with automatic delimiter detection...")
    df = pd.read_csv(csv_path, sep=None, engine="python")
    print("[INFO] Columns detected:", list(df.columns))

    if target is None:
        target = df.columns[-1]
        print(f"[INFO] Auto-detected target column = {target}")

    if target not in df.columns:
        raise ValueError(f"Target column '{target}' not found in dataset!")

    y = df[target]
    X = df.drop(columns=[target])

    # Preprocess target based on model_type
    if model_type == "binary":
        # convert yes/no/true/false -> 1/0
        if y.dtype == object:
            y = y.astype(str).str.strip().str.lower()
            y = y.replace({"yes": 1, "no": 0, "true": 1, "false": 0})
        # ensure numeric
        y = y.astype(np.float32).values.reshape(-1, 1)

    elif model_type == "multiclass":
        # convert strings to categorical integer codes if needed
        if y.dtype == object:
            y = y.astype("category").cat.codes
        # ensure integer labels (sparse labels)
        y = y.astype(np.int32).values.reshape(-1, 1)

    elif model_type == "regression":
        # numeric continuous
        y = y.astype(np.float32).values.reshape(-1, 1)

    else:
        raise ValueError("Unsupported model_type for loader")

    # One-hot encode only categorical predictors (keep numeric as-is)
    cat_cols = X.select_dtypes(include=["object"]).columns
    if len(cat_cols) > 0:
        print("[INFO] One-hot encoding predictors:", list(cat_cols))
        X = pd.get_dummies(X, drop_first=True)
    else:
        print("[INFO] No categorical predictor columns to encode.")

    # convert to float32 (features)
    X = X.astype(np.float32)

    # Split
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=(test_size + val_size), random_state=42, stratify=y if model_type!="regression" else None
    )
    # second split into val/test
    val_ratio = val_size / (test_size + val_size)
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=(1 - val_ratio), random_state=42,
        stratify=y_temp if model_type!="regression" else None
    )

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

# =====================================================================
# 2) GET DENSE LAYERS
# =====================================================================
def get_dense_layers(model):
    return [layer for layer in model.layers if isinstance(layer, Dense)]

# =====================================================================
# 3) ACTIVATION + GRADIENT STATS  (SHAPE-SAFE)
# =====================================================================
def compute_activation_grad_stats(model, dense_layers, X, y, model_type,
                                  batch_size=64, max_batches=30):
    """
    Returns stats dict: {layer.name: (A_mean_per_neuron, G_mean_per_neuron, Var_per_neuron)}
    Uses appropriate loss for model_type when computing gradients.
    """
    # Convert X,y to numpy if DataFrame
    if isinstance(X, pd.DataFrame):
        X_np = X.to_numpy()
    else:
        X_np = X
    if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
        y_np = y.to_numpy()
    else:
        y_np = y

    ds = tf.data.Dataset.from_tensor_slices((X_np, y_np)).batch(batch_size)
    acc_A = {l.name: [] for l in dense_layers}
    acc_G = {l.name: [] for l in dense_layers}
    acc_V = {l.name: [] for l in dense_layers}

    # choose loss for gradient computation
    if model_type == "binary":
        loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False, reduction=tf.keras.losses.Reduction.NONE)
    elif model_type == "multiclass":
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction=tf.keras.losses.Reduction.NONE)
    elif model_type == "regression":
        loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    else:
        raise ValueError("Unknown model_type for stats")

    batch_count = 0
    for xb, yb in ds:
        if batch_count >= max_batches:
            break
        batch_count += 1

        # ensure shapes consistent
        x = tf.cast(xb, tf.float32)
        # For sparse multiclass loss, yb should be shape (batch,) ints
        if model_type == "multiclass":
            yb_proc = tf.cast(tf.squeeze(yb, axis=-1), tf.int32)
        else:
            # binary/regression: keep shape (batch,1) as float32
            yb_proc = tf.cast(yb, tf.float32)

        layer_outputs = {}
        with tf.GradientTape(persistent=True) as tape:
            for layer in model.layers:
                x = layer(x)
                if layer in dense_layers:
                    tape.watch(x)
                    layer_outputs[layer.name] = x
            preds = x
            # compute per-sample loss (shape [batch,] or [batch, ...])
            per_sample_loss = loss_fn(yb_proc, preds)
            # compute scalar loss for gradient direction
            loss = tf.reduce_mean(per_sample_loss)

        for layer in dense_layers:
            name = layer.name
            if name not in layer_outputs:
                continue
            a = layer_outputs[name]            # shape (batch, units)
            A = tf.reduce_mean(tf.abs(a), axis=0).numpy()
            V = tf.math.reduce_variance(a, axis=0).numpy()

            g = tape.gradient(loss, a)
            if g is None:
                G = np.zeros_like(A)
            else:
                G = tf.reduce_mean(tf.abs(g), axis=0).numpy()

            acc_A[name].append(A)
            acc_G[name].append(G)
            acc_V[name].append(V)

        del tape

    # Aggregate
    stats = {}
    for layer in dense_layers:
        name = layer.name
        if len(acc_A[name]) == 0:
            stats[name] = (np.array([]), np.array([]), np.array([]))
            continue
        A = np.mean(np.stack(acc_A[name], axis=0), axis=0)
        G = np.mean(np.stack(acc_G[name], axis=0), axis=0)
        V = np.mean(np.stack(acc_V[name], axis=0), axis=0)
        stats[name] = (A, G, V)

    return stats

# =====================================================================
# 4) IMPORTANCE MASKS (protect last Dense layer)
# =====================================================================
def compute_importance_mask(
        stats,
        keep_ratio=0.7,
        alpha=0.5,
        beta=0.3,
        gamma=0.2,
        last_dense_name=None,
        min_units=4  # <-- increase minimum neurons per layer
    ):
    
    masks = {}
    for name, (A, G, V) in stats.items():
        if A.size == 0:
            masks[name] = np.array([], dtype=np.float32)
            continue

        def normalize(x):
            x2 = x - np.min(x)
            mx = np.max(x2)
            return x2 / (mx + 1e-12)

        # compute score
        score = alpha * normalize(A) + beta * normalize(G) + gamma * normalize(V)

        # ---- NEVER prune final Dense layer ----
        if last_dense_name is not None and name == last_dense_name:
            masks[name] = np.ones_like(score, dtype=np.float32)
            continue

        # ---- compute K ----
        k = int(len(score) * keep_ratio)
        k = max(k, min_units)        # enforce minimum
        k = min(k, len(score))       # safety

        # ---- If all scores are zero → randomly pick k neurons ----
        if np.all(score == 0):
            sel = np.random.choice(len(score), size=k, replace=False)
            mask = np.zeros_like(score)
            mask[sel] = 1
            masks[name] = mask.astype(np.float32)
            continue

        # normal path
        thresh = np.partition(score, -k)[-k]
        mask = (score >= thresh).astype(np.float32)

        # final safety: ensure >= min_units survive
        if np.sum(mask) < min_units:
            topk = np.argsort(score)[-min_units:]
            mask = np.zeros_like(score)
            mask[topk] = 1

        masks[name] = mask.astype(np.float32)

    return masks

# =====================================================================
# 5) DNAGate Layer (Zeroing)
# =====================================================================
# =====================================================================
# 5) DNAGate Layer (Zeroing) — NOW FULLY SERIALIZABLE
# =====================================================================
class DNAGate(tf.keras.layers.Layer):
    def __init__(self, mask=None, **kwargs):
        super().__init__(**kwargs)
        # store original numpy mask (for config saving)
        self._mask_np = None
        if mask is not None:
            self._mask_np = np.array(mask, dtype=np.float32)

        # keras-serializable field (converted to list)
        self.mask_list = self._mask_np.tolist() if self._mask_np is not None else None
        self.mask = None

    def build(self, input_shape):
        if self.mask_list is None:
            self.mask = None
        else:
            # rebuild tensor mask
            mask_np = np.array(self.mask_list, dtype=np.float32)
            if input_shape[-1] != mask_np.shape[0]:
                raise ValueError(
                    f"DNAGate mask length {mask_np.shape[0]} != layer units {input_shape[-1]}"
                )
            self.mask = tf.constant(mask_np, dtype=self.dtype)
        super().build(input_shape)

    def call(self, x):
        if self.mask is None:
            return x
        return x * self.mask

    # ⭐⭐ IMPORTANT — MAKES THE LAYER SAVEABLE ⭐⭐
    def get_config(self):
        config = super().get_config()
        config.update({
            "mask": self.mask_list,   # store mask as list
        })
        return config

    @classmethod
    def from_config(cls, config):
        # mask list gets restored automatically
        return cls(**config)

def build_masked_model(orig_model, masks):
    inp = Input(shape=orig_model.input_shape[1:])
    x = inp
    for layer in orig_model.layers:
        x = layer(x)
        if layer.name in masks:
            x = DNAGate(mask=masks[layer.name], name=layer.name + "_dna")(x)
    new_model = Model(inp, x)
    # copy weights where possible
    for l in orig_model.layers:
        try:
            new_model.get_layer(l.name).set_weights(l.get_weights())
        except Exception:
            pass
    return new_model

# =====================================================================
# 6) FLOPS + INFERENCE TIME
# =====================================================================
def dense_flops(in_size, out_size, bias=True):
    # multiply-adds counted as 2 ops per MAC
    return int(in_size * out_size * 2 + (out_size if bias else 0))

def model_flops(model):
    total = 0
    in_size = int(model.input_shape[1])
    for layer in model.layers:
        if isinstance(layer, Dense):
            total += dense_flops(in_size, layer.units, layer.use_bias)
            in_size = layer.units
    return total

def effective_flops(model, masks):
    total = 0
    in_size = int(model.input_shape[1])
    for layer in model.layers:
        if isinstance(layer, Dense):
            if layer.name in masks and masks[layer.name].size > 0:
                units = int(np.sum(masks[layer.name]))
            else:
                units = layer.units
            total += dense_flops(in_size, units, layer.use_bias)
            in_size = units
    return total

def flops_to_gflops(f):
    return f / 1e9

def flops_to_mflops(f):
    return f / 1e6

def measure_inference_time(model, X, runs=200):
    """Return average inference time in milliseconds per sample.
       Accepts X as pandas DataFrame or numpy array."""
    if isinstance(X, pd.DataFrame):
        X_np = X.to_numpy()
    else:
        X_np = X
    if len(X_np) == 0:
        return float("nan")
    # pick random indices
    idx = np.random.randint(0, len(X_np), size=min(runs, len(X_np)))
    samples = X_np[idx]
    # warm-up single predict
    model.predict(samples[:1], verbose=0)
    t0 = time.time()
    model.predict(samples, verbose=0)
    t1 = time.time()
    total_ms = (t1 - t0) * 1000.0
    return total_ms / len(samples)

# =====================================================================
# 7) STRUCTURAL PRUNING (REMOVE NEURONS) — protect final Dense layer
# =====================================================================
def structurally_prune_fnn(orig_model, masks, last_dense_name=None):
    inp = Input(shape=orig_model.input_shape[1:])
    x = inp
    prev_sel = None

    for layer in orig_model.layers:
        if isinstance(layer, Dense):
            W, b = layer.get_weights()

            # If previous layer was pruned → prune rows
            if prev_sel is not None:
                W = W[prev_sel, :]

            # ---- SAFETY: determine neurons to keep ----
            if last_dense_name is not None and layer.name == last_dense_name:
                # never prune final layer
                sel = np.arange(layer.units)

            else:
                mask = masks.get(layer.name, None)
                if mask is not None and mask.size > 0:
                    sel = np.where(mask == 1)[0]

                    # 💥 SAFETY FIX: ensure >=1 neuron survives
                    if len(sel) == 0:
                        sel = np.array([np.argmax(mask)])  
                else:
                    sel = None

            # ---- Apply pruning ----
            if sel is not None:
                W_new = W[:, sel]
                b_new = b[sel]
                new_units = len(sel)
            else:
                W_new = W
                b_new = b
                new_units = W.shape[1]

            new_layer = Dense(new_units, activation=layer.activation, name=layer.name)
            x = new_layer(x)
            new_layer.set_weights([W_new, b_new])

            prev_sel = sel

        else:
            x = layer(x)

    return Model(inp, x)

# =====================================================================
# 8) MAIN PIPELINE (auto-detect model type)
# =====================================================================
def fnn_pruning_pipeline(model_path, dataset_path,
                         keep_ratio=0.7,
                         alpha=0.5, beta=0.3, gamma=0.2,
                         calib_batches=30, batch_size=64, ft_epochs=3):
    # 1) load model and detect type
    print("Loading model:", model_path)
    model = load_model(model_path)
    model_type = detect_model_type(model)
    print(f"[INFO] Detected model type: {model_type}")

    loss_name, metrics = get_loss_and_metrics_for_type(model_type)
    print(f"[INFO] Using loss='{loss_name}', metrics={metrics}")

    # 2) load dataset adapted to model_type
    print("Loading dataset:", dataset_path)
    (X_train, y_train), (X_val, y_val), (X_test, y_test) = load_any_csv_dataset(dataset_path, model_type=model_type)

    # 3) find dense layers and the last Dense layer name
    dense_layers = get_dense_layers(model)
    last_dense_name = dense_layers[-1].name if len(dense_layers) > 0 else None
    print("Dense layers:", [l.name for l in dense_layers], "| last_dense:", last_dense_name)

    # 4) compute stats (activation + gradient)
    print("Computing activation & gradient stats...")
    stats = compute_activation_grad_stats(model, dense_layers, X_train, y_train, model_type,
                                          batch_size=batch_size, max_batches=calib_batches)

    # 5) compute masks (protect final layer)
    masks = compute_importance_mask(stats, keep_ratio=keep_ratio, alpha=alpha, beta=beta, gamma=gamma, last_dense_name=last_dense_name)
    for k,v in masks.items():
        if v.size > 0:
            print(f"{k} -> kept units: {int(np.sum(v))}/{len(v)}")

    # 6) build masked model (DNAGate zeroing) and compile with correct loss/metrics
    print("Building masked model (DNAGate zeroing)...")
    masked_model = build_masked_model(model, masks)
    masked_model.compile(optimizer='adam', loss=loss_name, metrics=metrics)

    # compute baseline & masked flops/accuracy
    baseline_acc = None
    masked_acc_before = None
    if model_type == "regression":
        baseline_eval = model.evaluate(X_val, y_val, verbose=0)
        baseline_acc = baseline_eval[0] if len(baseline_eval)>0 else None
        masked_acc_before = masked_model.evaluate(X_val, y_val, verbose=0)[0]
    else:
        baseline_acc = model.evaluate(X_val, y_val, verbose=0)[1]  # accuracy or sparse_accuracy
        masked_acc_before = masked_model.evaluate(X_val, y_val, verbose=0)[1]

    baseline_flops = model_flops(model)
    masked_flops = effective_flops(model, masks)

    print(f"Baseline Acc: {baseline_acc} | FLOPS: {baseline_flops}")
    print(f"Masked Acc (before FT): {masked_acc_before} | Effective FLOPS: {masked_flops}")

    # 7) fine-tune masked model
    print("Fine-tuning masked model...")
    masked_model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=ft_epochs, verbose=2)

    # evaluate masked after FT
    if model_type == "regression":
        masked_eval = masked_model.evaluate(X_val, y_val, verbose=0)
        masked_acc_after = masked_eval[0] if len(masked_eval)>0 else None
    else:
        masked_acc_after = masked_model.evaluate(X_val, y_val, verbose=0)[1]

    # 8) structural pruning (permanent neuron removal)
    print("Constructing structurally-pruned model (remove neurons)...")
    pruned_model = structurally_prune_fnn(model, masks, last_dense_name=last_dense_name)
    pruned_model.compile(optimizer='adam', loss=loss_name, metrics=metrics)

    print("Fine-tuning pruned model...")
    pruned_model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=ft_epochs, verbose=2)

    # final evals
    if model_type == "regression":
        pruned_eval = pruned_model.evaluate(X_val, y_val, verbose=0)
        pruned_acc = pruned_eval[0] if len(pruned_eval)>0 else None
    else:
        pruned_acc = pruned_model.evaluate(X_val, y_val, verbose=0)[1]

    pruned_flops = model_flops(pruned_model)

    # compute GFLOPS / MFLOPS / inference times
    baseline_gflops = flops_to_gflops(baseline_flops)
    masked_gflops = flops_to_gflops(masked_flops)
    pruned_gflops = flops_to_gflops(pruned_flops)

    baseline_mflops = flops_to_mflops(baseline_flops)
    masked_mflops = flops_to_mflops(masked_flops)
    pruned_mflops = flops_to_mflops(pruned_flops)

    baseline_time_ms = measure_inference_time(model, X_val)
    masked_time_ms = measure_inference_time(masked_model, X_val)
    pruned_time_ms = measure_inference_time(pruned_model, X_val)

    print("\n=== FINAL REPORT ===")
    print(f"Model type: {model_type}")
    print(f"Baseline:   Acc/metric={baseline_acc} | FLOPS={baseline_flops} | MFLOPS={baseline_mflops:.6f} | GFLOPS={baseline_gflops:.10f} | Time={baseline_time_ms:.4f} ms/sample")
    print(f"Masked:     Acc/metric={masked_acc_after} | EFLOPS={masked_flops} | MFLOPS={masked_mflops:.6f} | GFLOPS={masked_gflops:.10f} | Time={masked_time_ms:.4f} ms/sample")
    print(f"Pruned:     Acc/metric={pruned_acc} | FLOPS={pruned_flops} | MFLOPS={pruned_mflops:.6f} | GFLOPS={pruned_gflops:.10f} | Time={pruned_time_ms:.4f} ms/sample")
    if baseline_flops > 0:
        print(f"FLOPS reduction (pruned): {(baseline_flops - pruned_flops)/baseline_flops:.2%}")
    print("==============================================\n")

    print(f"Pruned:     Acc/metric={pruned_acc} | FLOPS={pruned_flops} | MFLOPS={pruned_mflops:.6f} | GFLOPS={pruned_gflops:.10f} | Time={pruned_time_ms:.4f} ms/sample")

    # ---- NEW METRIC REDUCTIONS ADDED BELOW ----
    if baseline_flops > 0:
        flop_reduction = (baseline_flops - pruned_flops) / baseline_flops
        gflop_reduction = (baseline_gflops - pruned_gflops) / baseline_gflops if baseline_gflops > 0 else 0
    else:
        flop_reduction = 0
        gflop_reduction = 0

    acc_reduction = baseline_acc - pruned_acc if baseline_acc is not None else None

    print(f"FLOPS reduction (pruned): {flop_reduction:.2%}")
    print(f"GFLOPS reduction (pruned): {gflop_reduction:.2%}")
    print(f"Accuracy reduction: {acc_reduction}")
    # ---- END NEW LINES ----

    print("==============================================\n")

    # save masked/pruned models next to original
    base = os.path.basename(model_path)
    root = os.path.splitext(base)[0]
    folder = os.path.dirname(model_path) or "."

    masked_path = os.path.join(folder, root + "_masked.h5")
    pruned_path = os.path.join(folder, root + "_pruned.h5")

    try:
        print(f"[INFO] Saving masked model to: {masked_path}")
        masked_model.save(masked_path)
    except Exception as e:
        print(f"[WARN] Failed to save masked model: {e}")

    try:
        print(f"[INFO] Saving pruned model to: {pruned_path}")
        pruned_model.save(pruned_path)
    except Exception as e:
        print(f"[WARN] Failed to save pruned model: {e}")

    # return X_val for further external checks if desired
    return model, masked_model, pruned_model, masks, X_val

# =====================================================================
# 9) RUN (example)
# =====================================================================
if __name__ == "__main__":
    # change these to your files
    model_path = r"D:\college\sem-8\models\dementia_fnn_model.h5"
    dataset_path = r"D:\college\sem-8\dataset\clean_dementia_dataset.csv"

    original_model, masked_model, pruned_model, masks, X_val = fnn_pruning_pipeline(
        model_path=model_path,
        dataset_path=dataset_path,
        keep_ratio=0.8,
        alpha=0.5, beta=0.3, gamma=0.2,
        calib_batches=30, batch_size=64, ft_epochs=15
    )
